In [ ]:
suppressMessages(library(SeuratDisk))
suppressMessages(library(SingleR))
suppressMessages(library(Seurat))
suppressMessages(library(SingleCellExperiment))
suppressMessages(library(UCell))

In [ ]:
dataset <- readRDS("../../int_data/dataset_ucb_qc.rds")

In [ ]:
data <- SingleCellExperiment(assays = list(counts = dataset@assays@data[["soupX_counts"]]),
                             colData = dataset@colData)
rownames(data) = rownames(dataset)

In [ ]:
data <- as.Seurat(data, data = NULL)

In [ ]:
data

In [ ]:
data <- NormalizeData(data)
data <- ScaleData(data, features = row.names(data))
data <- FindVariableFeatures(data)
data <- RunPCA(data, verbose = F)
data <- FindNeighbors(data, verbose = F)
data <- FindClusters(data, verbose = F)
data <- RunUMAP(data, dims = 1:20, verbose = F)

In [ ]:
ref1 <- celldex::BlueprintEncodeData()
ref2 <- celldex::MonacoImmuneData()
cell_classification <- SingleR(test = as.SingleCellExperiment(data),
                               ref = list(BPE = ref1, MON = ref2),
                               labels = list(ref1$label.main, ref2$label.main))

data[["SingleR_label"]] <- cell_classification$labels

plotScoreHeatmap(cell_classification)

In [ ]:
write.csv(data@meta.data, "dataset_ucb_metadata.csv")